# Model-based versus model-free learning project: $Q$-learning
<font color=red>Not much here.</font>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

## Problem 1

### Part A: Define the task

The task is composed of three possible states and two actions per state. Transition to a new state only occurs from the first stage of the task. Write a function <b><code>next_state</code></b> that returns the next state (either 1 or 2) based on the action taken (<code>LEFT</code> or <code>RIGHT</code>) in stage 1.

In [ ]:
LEFT = 0
RIGHT = 1

COMMON = 0
RARE = 1

def next_state(action):
    # write your code here
    # return a tuple that is the next state (1 or 2) and whether the
    # transition was common or rare.
    #    e.g. return (1, COMMON)
    

### Part B: Define the rewards

Reward earned on the task is determined by the action taken on stage 2 with reward probability consistently changing between .25 and .75 according to Gaussian random walks (with $\sigma$=.025).

Create an array that is of size 3 $\times$ 2 $\times$ 300 (number of states $\times$ number of actions $\times$ number of trials) where each sub-array, <code>R(s,a,:)</code>, is determined by a Gaussian random walk. If the random walk caused probabilities to leave the [.25, .75] range, then reflect the value around the minimum or maximum. For example, if the random walk would change <code>p</code> to <code>.20</code>, then its new value should be <code>p = .25 + (.25 - .2) = .3</code>. Likewise, is <code>p</code> would change to <code>0.87</code> then its new value should be <code>p = .75 - (.87 - .75) = .63</code>.

In [ ]:
R = np.zeros( (3,2,300) )

# randomly set intial reward probabilities between .25 and .75 for states 1, 2
# leave reward for stage 1 (state=0) zero
R[1:,:,0] = np.random.rand(2,2) * .5 + .25

for trial in range(1, 300):
    # write algorithm to create random walk for each state-action
    

plt.rcParams['figure.figsize'] = (10,5)
(figure, axis) = plt.subplots()
axis.plot(range(1,301), R[1,0,:], 'b-')
axis.plot(range(1,301), R[1,1,:], 'g-')
axis.plot(range(1,301), R[2,0,:], 'r-')
axis.plot(range(1,301), R[2,1,:], 'c-')
plt.ylim(0,1)
plt.xlabel('Trial #')
plt.ylabel('Probability of Reward')

plt.show()

### Part C: Softmax decision function

Choices in the task are made using a softmax decision function that assigns a probability of selecting an action that depends on the $Q$ value for that action relative to the alternative $Q$ value. Plot the softmax function for $Q$ values ranging from 0 to 1 assuming that the alternative action has $Q=0.5$.

In [ ]:
# inverse temperature parameter
tau = 10

def softmax(Q, Qalt):
    # write algorithm to calculate softmax probability

Qlist = np.arange(0, 1.05, .05)

# create a list named Ps that give the choice probability for each Q value in Qlist


plt.rcParams['figure.figsize'] = (5,5)
(figure, axis) = plt.subplots()
axis.plot(Qlist, Ps, 'k-')
axis.plot([0,1], [.5,.5], 'k:', linewidth=.25)
axis.plot([.5,.5], [1,0], 'k:', linewidth=.25)
plt.xlabel('Q')
plt.ylabel('Softmax Probability')
plt.show()

### Part D: SARSA algorithm

In order to reproduce the results in Otto et al. an additional element must be added to the SARSA algorithm described in Glasher et al. Specifically, $Q$ values from stage 1 must be updated based on prediction errors calculated on stage 2. This is a common feature of reinforcement learning algorithms that allows prior state-actions to gain some credit for reward earned later on.

Of course, early actions should not necessarily receive full credit for reward earned later. The way this is implemented in reinforcement learning is to define a separate variable $\lambda$ (called the <b>eligibility trace</b> such that prior actions are updated proportional to $\lambda^t$ for prediction errors occurring $t$ time steps after the action was taken. For the two stage task, this implies that stage 1 state-actions should be increased by $\alpha \times \lambda \times \delta$ where $\alpha$ is the learning rate and $\delta$ is the prediction error calculated on stage 2. Otto et al. used $\lambda=1$ (i.e. full credit for stage 1 actions based on reward earned in stage 2).

<b>Complete the following code to implement a SARSA learning algorithm.</b> I find it easiest to code this model by first determining first and second stage states and actions, followed by Q value updating based on prediction errors.

In [ ]:
elig_trace = 1 # i.e. lambda
gamma = 1
alpha = .2
tau = 5

# store stage 2 actions for plotting
stage2_actions = []

# initialize Q values
Q = np.zeros( (3,2) ) # i.e. number of states x number of actions

for trial in range(300):
    '''
    STAGE 1: determine action taken, whether the transition was 
    common or rare, and what the next state is.
    '''

        
    '''
    STAGE 2: determine action taken and whether reward was earned.
    Store stage 2 state in variable s2 (value of 1 or 2) and stage
    2 action in variable a2 (value of LEFT or RIGHT)
    '''

    
    '''
    STAGE 1 value update: determine prediction error and update
    Q value for transition from stage 1 to stage 2
    '''

    
    '''
    STAGE 2 value update: determine prediction error and update
    Q values for transition from stage 1 to stage 2. Recall that
    both stage 1 and stage 2 Q values must be updated based on
    stage 2 prediction error
    '''

    
    '''
    Store action taken on stage 2
    '''
    stage2_actions.append(2*(s2-1)+a2) # converts state-actions to (0,1,2,3)
    
'''
Plot the model performance
'''
plt.rcParams['figure.figsize'] = (10,10)
axis1 = plt.subplot(2,1,1)
a10 = np.add(np.where(np.asarray(stage2_actions)==0),1)
a11 = np.add(np.where(np.asarray(stage2_actions)==1),1)
a20 = np.add(np.where(np.asarray(stage2_actions)==2),1)
a21 = np.add(np.where(np.asarray(stage2_actions)==3),1)
axis1.plot(a10, 0*np.divide(a10,a10), 'b.')
axis1.plot(a11, 1*np.divide(a11,a11), 'g.')
axis1.plot(a20, 2*np.divide(a20,a20), 'r.')
axis1.plot(a21, 3*np.divide(a21,a21), 'c.')
plt.yticks((0,1,2,3), ('1,L', '1,R', '2,L', '2,R'))
plt.ylim(-.5,3.5)
plt.ylabel('Stage 2 Action')
plt.title('Actions with Drifting Reward Probabilities')

axis2 = plt.subplot(2,1,2)
axis2.plot(range(1,301), R[1,0,:], 'b-')
axis2.plot(range(1,301), R[1,1,:], 'g-')
axis2.plot(range(1,301), R[2,0,:], 'r-')
axis2.plot(range(1,301), R[2,1,:], 'c-')
plt.ylim(0,1)
plt.xlabel('Trial #')
plt.ylabel('Probability of Reward')

plt.show()

### Part E

<b>Copy and modify your code from Part D to reproduce Figure 1B from Otto et al.</b> 

To do this you need to store the probabilities of repeating the action taken in stage 1 based on whether the transition was <code>COMMON</code> or <code>RARE</code> and whether reward was earned on stage 2. Store these probabilities in the four arrays initialized below.

In [ ]:
stay_common_reward = []
stay_rare_reward = []
stay_common_noreward = []
stay_rare_noreward = []

elig_trace = 1 # i.e. lambda
gamma = 1
alpha = .2
tau = 5

# initialize Q values
Q = np.zeros( (3,2) ) # i.e. number of states x number of actions

for trial in range(300):
    '''
    STAGE 1: determine action taken, whether the transition was 
    common or rare, and what the next state is.
    '''

        
    '''
    STAGE 2: determine action taken and whether reward was earned.
    Store stage 2 state in variable s2 (value of 1 or 2) and stage
    2 action in variable a2 (value of LEFT or RIGHT)
    '''

    
    '''
    STAGE 1 value update: determine prediction error and update
    Q value for transition from stage 1 to stage 2
    '''

    
    '''
    STAGE 2 value update: determine prediction error and update
    Q values for transition from stage 1 to stage 2. Recall that
    both stage 1 and stage 2 Q values must be updated based on
    stage 2 prediction error
    '''

    
    '''
    Store stay probabilities in each of the 4 arrays 
    stay_common_reward, stay_rare_reward, stay_common_noreward, stay_rare_noreward
    '''


plt.rcParams['figure.figsize'] = (10,7)
(figure, axis) = plt.subplots()
axis.bar((1,3), (np.mean(stay_common_reward), np.mean(stay_common_noreward)), .75, color='blue')
axis.bar((2,4), (np.mean(stay_rare_reward), np.mean(stay_rare_noreward)), .75, color='red')
plt.ylim(.25,1)
plt.ylabel('Stay Probability')
axis.set_xticklabels(('Rewarded', 'Unrewarded'))
axis.set_xticks((1.5, 3.5))
plt.legend(('Common', 'Rare'))
plt.title('Model Free')

plt.show()

## Problem 2

The following code defines and draws a maze. The green circle is the starting position (stored in the tuple <code>maze.start</code>) and the red circle is where reward is – and where running through the maze is terminated.

In [ ]:
class Maze:
    size_rows = 5
    size_cols = 5
    
    start = (0,0)
    
    # 4 possible actions: 0-UP, 1-RIGHT, 2-DOWN, 3-LEFT
    actions = np.zeros( (size_rows, size_cols, 4) )
    # reward for each state is defined in init_rewards
    reward = np.zeros( (size_rows, size_cols) )
    
    def get_actions(self, state):
        action_array = self.actions[state[0], state[1]]
        return [i for i in range(len(action_array)) if action_array[i]]
    
    def get_reward(self, state):
        return self.reward[state[0], state[1]]
    
    def next_state(self, state, action):
        a = self.get_actions((state[0], state[1]))
        if action in a:
            state_delta = [ (0,1), (1,0), (0,-1), (-1,0) ]
            return (state[0] + state_delta[action][0], state[1] + state_delta[action][1])
        else:
            return state
        
    def draw_with_path(self, path, title=''):
        plt.rcParams['figure.figsize'] = (5,5)
        (figure, axis) = plt.subplots()
        for i in range(self.size_rows):
            for j in range(self.size_cols):
                if (i,j) == self.start:
                    circle = plt.Circle((i+.5, j+.5), radius=0.25, fc='g')
                    axis.add_patch(circle)
                    
                if self.reward[i,j]:
                    circle = plt.Circle((i+.5, j+.5), radius=0.25, fc='r')
                    axis.add_patch(circle)
                
                a = self.actions[i,j]
                if not a[0]: # UP
                    axis.plot([i, i+1], [j+1, j+1], 'k-')
                if not a[1]: # RIGHT
                    axis.plot([i+1,i+1], [j, j+1], 'k-')
                if not a[2]: # DOWN
                    axis.plot([i,i+1], [j, j], 'k-')
                if not a[3]: # LEFT
                    axis.plot([i,i], [j, j+1], 'k-')
        
        plt.Circle((.5, .5), radius=0.25, fc='g')
        
        r1 = (0.5,0.5)
        for i in range(1, len(path)):
            r2 = np.random.uniform(.3,.7,2) if i<len(path)-1 else (.5,.5)
            s1 = np.add(path[i-1], r1)
            s2 = np.add(path[i], r2)
            r1 = r2
            
            axis.plot([s1[0], s2[0]], [s1[1], s2[1]], 'm-.')
        
        if len(title):
            plt.title(title)
        
        plt.show()
      
    def draw(self):
        self.draw_with_path([])
    
    def init_actions(self):
        # first, make all actions illegal
        self.actions = np.zeros( (self.size_rows,self.size_cols,4) )
        
        # now let's draw a maze
        self.actions[0,0,0] = 1
        self.actions[0,1,2] = 1
        self.actions[0,1,0] = 1
        self.actions[0,1,1] = 1
        self.actions[0,2,2] = 1
        self.actions[1,1,3] = 1
        self.actions[1,1,1] = 1
        self.actions[2,1,3] = 1
        self.actions[2,1,1] = 1
        self.actions[3,1,3] = 1
        self.actions[3,1,2] = 1
        self.actions[3,0,0] = 1
        self.actions[3,0,3] = 1
        self.actions[2,0,1] = 1
        self.actions[2,0,3] = 1
        self.actions[1,0,1] = 1
        self.actions[2,1,0] = 1
        self.actions[2,2,2] = 1
        self.actions[2,2,0] = 1
        self.actions[2,3,2] = 1
        self.actions[2,3,1] = 1
        self.actions[2,3,3] = 1
        self.actions[1,3,1] = 1
        self.actions[1,3,2] = 1
        self.actions[1,2,0] = 1
        self.actions[3,3,3] = 1
        self.actions[3,3,1] = 1
        self.actions[4,3,3] = 1
        self.actions[4,3,2] = 1
        self.actions[4,2,0] = 1
        self.actions[4,2,3] = 1
        self.actions[3,2,1] = 1
        self.actions[4,2,2] = 1
        self.actions[4,1,0] = 1
        self.actions[4,1,2] = 1
        self.actions[4,0,0] = 1
        self.actions[1,3,0] = 1
        self.actions[1,4,2] = 1
        self.actions[1,4,3] = 1
        self.actions[0,4,1] = 1
        self.actions[0,4,2] = 1
        self.actions[0,3,0] = 1
        self.actions[1,4,1] = 1
        self.actions[2,4,3] = 1
        self.actions[2,4,1] = 1
        self.actions[3,4,3] = 1
        self.actions[3,4,1] = 1
        self.actions[4,4,3] = 1
    
    def init_rewards(self):
        self.reward = np.zeros( (self.size_rows, self.size_cols) )
        self.reward[self.size_rows-1,self.size_cols-1] = 1
    
    def __init__(self):
        self.init_actions()
        self.init_rewards()
        
maze = Maze()
maze.draw()

The maze code comes with a few functions that should hopefully make writing navigation code easier. The position within the maze should be stored as a tuple where <code>(0,0)</code> is the bottom left corner, moving to the right increases the first element of the tuple, and moving up increases the second element of the tuple. If you initialize a Maze object with

<code>maze = Maze()</code>

then <code>maze.start</code> gives you the intended start position within the maze.

For any state, <code>s</code>, within the maze, the function <code>maze.get_actions()</code> returns a list of all possible actions available from that state (i.e. the walls of the maze may not be crossed and hence represent illegal actions). You can determine the next state reached for action <code>a</code> in state <code>s</code> using <code>maze.next_state(s,a)</code>. The reward available in a state is obtained using <code>maze.get_reward(s)</code>.

To see how this all works, let's begin be defining a softmax decision function (see Glascher et al., 2010):

In [ ]:
# inverse temperature – adjust this as you see fit
tau = 10

def Ps(Q, Qall):
    numerator = math.exp(tau * Q)
    
    denominator = 0
    for i in range(len(Qall)):
        denominator += math.exp(tau * Qall[i])
    
    return numerator / denominator

We can then create a skeleton for a $Q$-learning model (that does not learn!) in the following way:

In [ ]:
# store Q-values based on x position, y position, and possible directions of movement
Q = np.zeros( (maze.size_rows, maze.size_cols, 4) )

# start at intended beginning location
s = maze.start
# store all traversed positions in maze in list for drawing at end
s_hist = [s]

# navigate until reward is reached
while not maze.get_reward(s):
    # find all possible actions from current position
    a_poss = maze.get_actions(s)

    # calculate Q-values for all actions from the current state
    Qall = [Q[s[0], s[1], a] for a in a_poss]

    # convert Q-values into probability of selection using softmax function
    Ps_all = [Ps(Q, Qall) for Q in Qall]
    
    # randomly select action based on probability of selection
    Ps_cumsum = np.cumsum(Ps_all)
    rand = np.random.uniform()
    for i in range(len(a_poss)):
        if rand <= Ps_cumsum[i]:
            a = a_poss[i]
            break

    # traverse to next state
    s = maze.next_state(s, a)
    # reward earned?
    r = maze.get_reward(s)

    # add new state to list of visited states
    s_hist.append(s)

# draw the maze with the random path
maze.draw_with_path(s_hist)   

### Part A

Write a SARSA algorithm that learns to navigate the maze. I found that using a learning rate of <code>alpha=0.3</code> and discount rate of <code>gamma=0.9</code> allows the maze to be learned in under 100 trials.

Recall that the prediction error in the SARSA algorithm is given by

$$\delta(s) = r(s') + \gamma Q(s',a') - Q(s,a)$$

where $s'$ and $a'$ are the state and action taken after action $a$ is taken in state $s$.

<b>Plot the number of actions required to reach the reward on 100 runs through the maze as a function of trial number.</b>

In [ ]:
Q = np.zeros( (maze.size_rows, maze.size_cols, 4) )
eta = 0.3
gamma = .9

step_counts = []

for trial in range(100):
    num_steps = 0
    
    s = maze.start
    s_hist = []
    
    while not maze.get_reward(s) and num_steps<500:
        num_steps += 1
        s_hist.append(s)
        
        # SARSA algorithm!
    
    s_hist.append(s)
    
    if trial == 0:
        s_hist_init = s_hist.copy()
    
    step_counts.append(num_steps)


maze.draw_with_path(s_hist_init, 'Initial')
maze.draw_with_path(s_hist, 'Final')
(figure,axis) = plt.subplots()
axis.plot(step_counts)
plt.xlabel('Trial #')
plt.ylabel('Steps to reward')
plt.title('Learning')
plt.show()

### Part B

Repeat Part A except using the Q-learning algorithm.

Recall that the prediction error in the Q-learning algorithm is given by

$$ \delta(s) = r(s') + \max_{a'} \gamma Q(s',a') - Q(s,a). $$

In [ ]:
Q = np.zeros( (maze.size_rows, maze.size_cols, 4) )
eta = 0.3
gamma = .9

step_counts = []

for trial in range(100):
    num_steps = 0
    
    s = maze.start
    s_hist = []
    
    while not maze.get_reward(s) and num_steps<500:
        num_steps += 1
        s_hist.append(s)
        
        # Q-learning algorithm!
    
    s_hist.append(s)
    
    if trial == 0:
        s_hist_init = s_hist.copy()
    
    step_counts.append(num_steps)


maze.draw_with_path(s_hist_init, 'Initial')
maze.draw_with_path(s_hist, 'Final')
(figure,axis) = plt.subplots()
axis.plot(step_counts)
plt.xlabel('Trial #')
plt.ylabel('Steps to reward')
plt.title('Learning')
plt.show()